In [7]:
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import decoupler as dc

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:
output_pb = "./submission/data/MOFAniche_pb_data.csv"
output_coldat = "./submission/data/MOFAniche_pb_coldata.csv"

In [3]:
meta = pd.read_csv('/Volumes/sd22b002/guest/VisiumMS/config/meta.csv')
vs_samples = meta[~meta['Batch vs'].isnull()]['Sample id'].values.astype('U')

# Process hallmarks and abunds
adata = []
for sample_id in vs_samples:
    slide = sc.read_h5ad('/Volumes/sd22b002/guest/VisiumMS/data/prc/vs/{0}/adata.h5ad'.format(sample_id))
    slide.obs['niches'] = pd.read_csv('/Volumes/sd22b002/guest/VisiumMS/data/prc/vs/{0}/niches.csv'.format(sample_id), index_col=0)
    slide.obs_names = [sample_id + '_' + i for i in slide.obs_names]
    slide = slide[~slide.obs['areas'].isnull()].copy()
    adata.append(slide)
adata = ad.concat(adata, join='outer')

In [9]:
padata = dc.get_pseudobulk(adata, 
                           sample_col='Sample id', 
                           groups_col='niches', 
                           layer='counts', 
                           min_prop=0, 
                           min_smpls=0,
                           mode='sum')

pb_dat = pd.DataFrame(padata.X)
pb_dat.columns = padata.var.index.values
pb_dat.index = padata.obs.index.values
pb_dat.to_csv(output_pb)

In [10]:
pb_coldata = padata.obs.copy()
pb_coldata["colname"] = pb_coldata.index.values
pb_coldata.to_csv(output_coldat)